## Convolutional Neural Network 
This is a framework for understanding the working of CNN network using the MNIST data
thanks ot Udacity Team for providing materials which made me understang the basics of DEEP learning and CNN networks

import the needed modules to run this program

In [56]:
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf
import numpy as np
import time

In [57]:
### I had problem in executing this line
#mnist = input_data.read_data_sets(".", one_hot=True, reshape=False)
#the above line will import the data in shape of (:,28,28,1)
#unfortunately i was not able to run this code in my system
#so i was forced to use this code , which gives the image data of shape (:,784)
#mnist = input_data.read_data_sets('/datasets/ud730/mnist', one_hot=True)
#this is the reason of converting the data shape . if you are able to import the image data of shape(28,28,1)
#then you dont have to run the conversion block of this program
###else , yu have to run those block


#mnist = input_data.read_data_sets(".", one_hot=True, reshape=False)
mnist = input_data.read_data_sets('/datasets/ud730/mnist', one_hot=True)


Extracting /datasets/ud730/mnist\train-images-idx3-ubyte.gz
Extracting /datasets/ud730/mnist\train-labels-idx1-ubyte.gz
Extracting /datasets/ud730/mnist\t10k-images-idx3-ubyte.gz
Extracting /datasets/ud730/mnist\t10k-labels-idx1-ubyte.gz


### Storage place
create a place to store the weights and bias after running the training module . this data can be used to classify images of the MNIST data later without having to re-run the training again

In [58]:
# create a location to save the data
save  = 'D:\\tutorials\\nanodegree\\SDC\\unit_5_CNN\\code\\saved_data\\convModel.ckpt'


In [80]:
img = mnist.train.images
labels = mnist.train.labels
validImg = mnist.validation.images
validLables = mnist.validation.labels
testImg = mnist.test.images
testLabels = mnist.test.labels

In [77]:
learning_rate = 0.00001
n_classes = 10
epochs = 10
batch_size = 128 # change this if you have a good system (256, 512 ,1024 ,...)
test_valid_size = 10
dropout = 0.75

## Converting the shape of the train , test and validation data
convert the image data of shape (:,784) to (:,28,28,1) . this is the shape on which we are going to apply our CNN 

you dont have to do this if you are using 

mnist = input_data.read_data_sets(".", one_hot=True, reshape=False)


In [81]:
if img.shape[1] == 784:
    img = np.reshape(img,(-1,28,28,1))
    labels = np.reshape(labels,(-1,10))
    validImg = np.reshape(validImg,(-1,28,28,1))
    validLables = np.reshape(validLables,(-1,10))
    testImg = np.reshape(testImg,(-1,28,28,1))
    testLabels = np.reshape(testLabels,(-1,10))
    #print (labels.shape)
    print ('all the data is converted ')
    print ('the new shapes are :')
    print ('Train image {}'.format(img.shape))
    print ('Test image {}'.format(testImg.shape))
    print ('Validation image {}'.format(validImg.shape))
else:
    print (' you dont have to convert it .. your data looks fine : ) ')

all the data is converted 
the new shapes are :
Train image (55000, 28, 28, 1)
Test image (10000, 28, 28, 1)
Validation image (5000, 28, 28, 1)


## Initialization of parameters
these are the parameters which we are going to use . feel free to change and play around with it

in my case , my laptop is not having a GPU build in and my RAM is just 6GB . i have restricted myself to 
batch_size of 128 . if you are running this on a higher config system , feel freeto change it (256,512,1024.....)

In [61]:
learning_rate = 0.00001
n_classes = 10
epochs = 10
batch_size = 128
test_valid_size = 10
dropout = 0.75

In [63]:

no_of_data = 100# change this number to have more or less testing and validation data
valid_x = validImg[0:no_of_data,:,:,:]
valid_y = validLables[0:no_of_data,:]
test_x  = testImg[0:no_of_data,:,:,:]
test_y  = testLabels[0:no_of_data,:]

### weights and biase 

In [64]:
weight = {'wc1':tf.Variable(tf.random_normal([5,5,1,32])),
          'wc2':tf.Variable(tf.random_normal([5,5,32,64])),
          'wd1':tf.Variable(tf.random_normal([7*7*64,1024])),
          'out':tf.Variable(tf.random_normal([1024,n_classes]))}

biases = {'bc1':tf.Variable(tf.random_normal([32])),
          'bc2':tf.Variable(tf.random_normal([64])),
          'bd1':tf.Variable(tf.random_normal([1024])),
          'out':tf.Variable(tf.random_normal([n_classes]))}


In [65]:
# create a saver to save all our work
saver = tf.train.Saver()

## Begining of CNN

In [66]:
# Create convolute layer
def conv2d (x,w,b,stride = 1):
    x = tf.nn.conv2d(x,w,strides=[1,stride,stride,1],padding='SAME')
    x = tf.nn.bias_add(x,b)
    return tf.nn.relu(x)

In [67]:
# Apply max pooling to the convolute layer
def maxpool2d(x,k):
    return tf.nn.max_pool(x,ksize = [1,k,k,1],strides = [1,k,k,1],padding = 'SAME')

In [68]:
# creation of the Graph
'''
Layers of our Graph
Input layer
convolution layer 1
convolution layer 2
fully connected layer 
output layer
'''
def conv_net(x,weight,biases,dropout):
    convnetLayer1 = conv2d(x,weight['wc1'],biases['bc1'])
    convnetLayer1 = maxpool2d(convnetLayer1,k=2)
    
    convnetLayer2 = conv2d(convnetLayer1,weight['wc2'],biases['bc2'])
    convnetLayer2 = maxpool2d(convnetLayer2,k=2)
    
    fc1 = tf.reshape(convnetLayer2, [-1, weight['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weight['wd1']), biases['bd1'])
    
    
    fc1 = tf.nn.relu(fc1)
    fc1 = tf.nn.dropout(fc1,dropout)
    out = tf.add(tf.matmul(fc1,weight['out']),biases['out'])
    
    
    return out
    

### Placeholder creation
create placeholder for 
* input image
* Lables
* Logits
* cost
* optimizer
* accuracy
* predictions

In [69]:

x = tf.placeholder(tf.float32,[None,28,28,1])
y = tf.placeholder(tf.float32,[None,n_classes])
keep_prob = tf.placeholder(tf.float32)
logits = conv_net(x,weight,biases,keep_prob)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = logits ,
                                                             labels = y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate = learning_rate)\
                .minimize(cost)
correct_pred = tf.equal(tf.argmax(logits,1),tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred,tf.float32))

### Execute our graph
Time to check our creation . run the block below to start training process and store the session onto a file specified

#### Time to complete 
 it took me over 2 hours to complete the process of training . so , be prepered to burn your waiting time while our network is training

In [70]:
start_time = time.time()
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    print ('stage 1 is initialized')
    for epoch in range(epochs):
            print ('{} epoch'.format(epoch))
            for batch in range(int(img.shape[0]/batch_size)):
                batch_x = img[0+(batch*batch_size):(batch*batch_size)+batch_size,:,:,:]
                batch_y = labels[0+(batch*batch_size):(batch*batch_size)+batch_size,:]
                
                
                sess.run(optimizer,feed_dict={x:batch_x,y:batch_y,keep_prob : dropout})
            
                loss=sess.run(cost,feed_dict={x:batch_x,y:batch_y,keep_prob : 1.0})
                if batch%100 ==0:
                    print ('the process is at {} epoch , {} batch'.format(epoch,batch))
                
            print ('time taken for execution of epoch {} is {}'.format(epoch,time.time()-start_time))
            print ('the loss in this calculation is : {}'.format(loss))
            
            # to test the validation of our model
            valid_acc = sess.run(accuracy,feed_dict={
                x: valid_x,
                y: valid_y,
                keep_prob: 1.})
            print('Epoch {:>2}, Batch {:>3} -'
                  'Loss: {:>10.4f} Validation Accuracy: {:.6f}'.format(
                epoch + 1,
                batch + 1,
                loss,
                valid_acc))
    # to print the total accuracy
    test_acc = sess.run(accuracy, feed_dict={x: test_x,
        y:test_y,
        keep_prob: 1.})
    print('Testing Accuracy: {}'.format(test_acc))
    print ('total time of execusion : {}'.format(time.time()-start_time))
    saver.save(sess,save)
    print('the process is saved correctly !!!')


stage 1 is initialized
0 epoch
the process is at 0 epoch , 0 batch
the process is at 0 epoch , 100 batch
the process is at 0 epoch , 200 batch
the process is at 0 epoch , 300 batch
the process is at 0 epoch , 400 batch
time taken for execution of epoch 0 is 584.0991532802582
the loss in this calculation is : 669.2898559570312
Epoch  1, Batch 429 -Loss:   669.2899 Validation Accuracy: 0.760000
1 epoch
the process is at 1 epoch , 0 batch
the process is at 1 epoch , 100 batch
the process is at 1 epoch , 200 batch
the process is at 1 epoch , 300 batch
the process is at 1 epoch , 400 batch
time taken for execution of epoch 1 is 2222.907201528549
the loss in this calculation is : 361.03076171875
Epoch  2, Batch 429 -Loss:   361.0308 Validation Accuracy: 0.810000
2 epoch
the process is at 2 epoch , 0 batch
the process is at 2 epoch , 100 batch
the process is at 2 epoch , 200 batch
the process is at 2 epoch , 300 batch
the process is at 2 epoch , 400 batch
time taken for execution of epoch 2 i

### finished 
hope this code worked without giving you hard time . 
my results were 
Epoch 10, Batch 429 -Loss:   163.1596 Validation Accuracy: 0.770000
Testing Accuracy: 0.8799999952316284
total time of execusion : 7785.77493929863

##### I like to see your model bet mine  . thanks !!!